In [2]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm import tqdm
from datetime import datetime


**Load the newspapers**

In [3]:
base_path = r'\\ent1files.epfl.ch\ElyseeMuseum\PROJECT_DATA\BCU_v2\RawData_newspaper\LLE_temp'
data = []

nb_news_tot = 0
nb_news = 0

def parse_alto_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    ns = {'alto': 'http://schema.ccs-gmbh.com/ALTO'}
    text_blocks = []
    for text_block in root.findall('.//alto:TextBlock', ns):
        block_text = ""
        for text_line in text_block.findall('.//alto:TextLine', ns):
            for string in text_line.findall('.//alto:String', ns):
                block_text += string.attrib['CONTENT'] + ' '
            block_text += '\n'
        text_blocks.append(block_text.strip())
    return text_blocks



newspapers_dict = {}

for newspaper_dir in tqdm(next(os.walk(base_path))[1], desc="Processing Newspapers"):
    alto_dir = os.path.join(base_path, newspaper_dir, "ALTO")
    nb_news_tot += 1
    if nb_news == 5: 
        break
    if os.path.exists(alto_dir):
        nb_news += 1
        alto_files = os.listdir(alto_dir)
        newspaper_blocks = []  # List to store text blocks for the current newspaper
        for alto_file in tqdm(alto_files, desc=f"Processing {newspaper_dir}"):
            if alto_file.endswith(".xml"):
                alto_file_path = os.path.join(alto_dir, alto_file)
                blocks = parse_alto_xml(alto_file_path)
                newspaper_blocks.extend(blocks)
        newspapers_dict[newspaper_dir] = newspaper_blocks
    else:
        print(f"ALTO directory not found for {newspaper_dir}")



Processing Newspapers:   0%|          | 5/2102 [00:06<48:27,  1.39s/it]


In [4]:
import json

# Save to a JSON file
with open('newspapers_blocks.json', 'w', encoding='utf-8') as f:
    json.dump(newspapers_dict, f, ensure_ascii=False, indent=4)


In [5]:
# Load the JSON file
with open('newspapers_blocks.json', 'r', encoding='utf-8') as f:
    newspapers_dict = json.load(f)


In [6]:
list(newspapers_dict.keys())

['19571025_01', '19330526_01', '19330527_01', '19330529_01', '19330530_01']

In [7]:
newspaper_key = "19330526_01" 
print(f"Text blocks for {newspaper_key}:")

for i, block in enumerate(newspapers_dict[newspaper_key], start=1):
    print(f"Block {i}:\n{block}\n\n---\n")




Text blocks for 19330526_01:
Block 1:
R 1 : DACTIONET 
ADMINISTRATION 
as . Avenue dePérolles 
TÉLÉPHONES 
Réùaclion ~ : 1309 
ALonnements : 3 . 79 
Comptepost . IlaM 
PBIXDESABUNNEMENTS 
Im . 3 rn 6 m . l . ! m . 
Suisse 2506 .- 9 .-18 . 
Etranger 4 . 5010 .-19 .-38 .-

---

Block 2:
NOUVELLESDUJOUR

---

Block 3:
Les · ridiculestéDloiosàdéchargeàGenève . 
LunificationdesEglisesprotestantesdAlleDlagne .

---

Block 4:
M diacommencé , àGenève , auxASSi- / 
ercre , . 
sesfédérales , ledéfilédestémoinscitéspar 
la défense . 1 1 . Dickeravaittriomphalement 
annoncé quil interrogerai ~ abondammel ~ t 
quelques-uns decesme ~ sleurs .. Onall ~ 
voirlLa « vérité » allait surgir , bondir , 
éclater silonosedire ! M . Dickersétait 
trompé , comme unsimpleNicoulaz annon 
cantlezrandsoir » , 
• Qt heuresdedépositions socialistesont 
uare 
marquéunetellerépétition desmêmespro 
pOS , brodéssurlemêmecanev ~ s , bref , ? nt 
donnéunetelleimpression dermseenscene 
teetfigéequelles ontfatigué ct 
monoo

**Functions to Process newspapers**

In [8]:
from nltk.tokenize import word_tokenize
import re
import textwrap



# Function to remove numbers, punctuation, and tokenize
def preprocess_and_tokenize(text):
    # Remove numbers
    text_no_numbers = re.sub(r'\d+', '', text)
    # Remove punctuation
    text_no_punctuation = re.sub(r'[^\w\s]', '', text_no_numbers)
    # Tokenize
    tokens = word_tokenize(text_no_punctuation)
    return tokens


def print_block(block, width=80):
    wrapper = textwrap.TextWrapper(width=width)
    
    formatted_passage = ' '.join(block)
    wrapped_passage = wrapper.fill(text=formatted_passage)
    print(wrapped_passage)
    print("\n---\n")

def print_text(text, width=80):
    wrapper = textwrap.TextWrapper(width=width)
    wrapped_passage = wrapper.fill(text=text)
    print(wrapped_passage)
    print("\n---\n")

    
    
    



In [9]:
processed_newspapers_dict = {}

for newspaper, blocks in newspapers_dict.items():
    processed_blocks = [preprocess_and_tokenize(block) for block in blocks]
    processed_newspapers_dict[newspaper] = processed_blocks


In [10]:
print_block(processed_newspapers_dict["19330526_01"][20])

Lesderniers témoins àcharge Genève mai M Cogneaentendu le novembre ledis
coursdeNicola discoursviolentdontilaretenu cettephrase Cettevieille prostituée
dar mée Unpeuplustard ilaégalement entendu unorateur dire Enfonçons lebarragel
Letémoinestassermenté Cognesetrouvait aucarrefour àlarrivée delatroupe
IIavuunmanifestant lancer poivreauxsoldats etàungardefrontière témoin
avudesindividus quifrappaient des soldats M Augustin Gaille agent depolice àLau
sanne aassistélejeudi lOnovembre àun meeting tenu àLausanne aucours duquel
LucienTronchet acommenté lesévénements dequ Genève Tronchet anotamment ditque
Pour unquiafaitduservice militaire ilnyavait pas à setromper surlasignification
duclairon etTronchet aajouté que poursoncompte Yil sétaitcachéderrière
uneautomobile MM DuflonetMullerontvulaccuséM Du pont marcher aupremier rang
dunecalonne demanifestants M Walther Vermouth faisait partie dela compagnie
Ilareçuuncoupdecasquesurlatêteet futhospitalisé douzejoursàlhôpital Unsoldat M
Emile Bouvier

**Part to select passages that contains words related to fribourg**

In [11]:
def filter_blocks(blocks, interesting_words):
    """
    Filters blocks to keep only those containing specific interesting words.
    Args:
    - blocks: A list of lists, where each inner list is a block of tokenized words.
    - interesting_words: A list of words considered interesting for filtering.

    Returns:
    - A list of blocks that contain any of the interesting words.
    """
    filtered_blocks = []
    for block in blocks:
        if any(word in block for word in interesting_words):
            filtered_blocks.append(block)
    return filtered_blocks


interesting_words = ["Fribourg", "fribourg", "fribourgeois", "fribourgeoise"]

filtered_newspapers_dict = {}
for newspaper, blocks in processed_newspapers_dict.items():
    filtered_blocks = filter_blocks(blocks, interesting_words)
    if filtered_blocks:
        filtered_newspapers_dict[newspaper] = filtered_blocks



In [12]:

nb_newspapers_with_block = len(filtered_newspapers_dict)
total_blocks = sum(len(blocks) for blocks in filtered_newspapers_dict.values())
avg_nb_block = int(total_blocks / nb_newspapers_with_block) if nb_newspapers_with_block > 0 else 0
nb_tokens = sum(sum(len(block) for block in blocks) for blocks in filtered_newspapers_dict.values())
avg_tokens_per_newspaper = int(nb_tokens / nb_newspapers_with_block) if nb_newspapers_with_block > 0 else 0

print(f"Nb newspaper having interesting word: {nb_newspapers_with_block} ~ {(nb_newspapers_with_block/nb_news)*100:.2f}%")
print(f"Average number of interesting blocks per newspaper: {avg_nb_block}")
print(f"Total number of tokens retained: {nb_tokens}")
print(f"Average number of tokens retained per newspaper: {avg_tokens_per_newspaper}")


Nb newspaper having interesting word: 5 ~ 100.00%
Average number of interesting blocks per newspaper: 18
Total number of tokens retained: 7056
Average number of tokens retained per newspaper: 1411


In [13]:
with open('filtered_blocks.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_newspapers_dict, f, ensure_ascii=False, indent=4)

with open('filtered_blocks.json', 'r', encoding='utf-8') as f:
    filter_newspapers_dict = json.load(f)

In [14]:
for block in filter_newspapers_dict[list(filter_newspapers_dict.keys())[0]]:
    print_block(block)

trouverait place tout de suite ou ä convenir chez Athanase Roos marechal et
cycles CHARMEY Gruyere Fribourg

---

ä M ra Descloux Cecile Simplon Fribourg

---

Fribourg TeL Arcades ffaro

---

Monsieur lAbbe Henri Ferrero k Chäbles Fribourg ainsi que les familles Ferrero
Speckel et parentes k Genöve et en Italie ont le chagrin de faire part du döces
de Monsieur

---

Ma on trfes qualifie nombreuses annees d expe rience demande travaux reparations
transforma tion carrelage travail ä lheure conviendrait prix modere Sadresser
Poste restante I sous CL Fribourg

---

Fribourg Fr avec immeuble locatif Materiel moderne Recet tes Fr an Agence
DESPONT Ruchonnet Lausanne

---

ancienne magnifiques peintures Les interesses peuvent sadresser par Gerit soui
chiffres F N au bureau dannonces des Frei burger Nachrichten avenue de la Gare
Fribourg

---

_ en laine double face ans Ö _ plus augmentation par taiile T Sr Timbres du
Service d escompte fribourgeois

---

une institution fribourgeoise

---

N

**OPENAI text correction**

In [15]:
import openai
from openai import OpenAI


In [16]:
client = OpenAI(api_key='YOUR_PRIVATE_KEY')


In [17]:
list_to_process = processed_newspapers_dict["19330526_01"][20]


text_to_process = " ".join(list_to_process)

print("Text before processing")
print_text(text_to_process)


Text before processing
Lesderniers témoins àcharge Genève mai M Cogneaentendu le novembre ledis
coursdeNicola discoursviolentdontilaretenu cettephrase Cettevieille prostituée
dar mée Unpeuplustard ilaégalement entendu unorateur dire Enfonçons lebarragel
Letémoinestassermenté Cognesetrouvait aucarrefour àlarrivée delatroupe
IIavuunmanifestant lancer poivreauxsoldats etàungardefrontière témoin
avudesindividus quifrappaient des soldats M Augustin Gaille agent depolice àLau
sanne aassistélejeudi lOnovembre àun meeting tenu àLausanne aucours duquel
LucienTronchet acommenté lesévénements dequ Genève Tronchet anotamment ditque
Pour unquiafaitduservice militaire ilnyavait pas à setromper surlasignification
duclairon etTronchet aajouté que poursoncompte Yil sétaitcachéderrière
uneautomobile MM DuflonetMullerontvulaccuséM Du pont marcher aupremier rang
dunecalonne demanifestants M Walther Vermouth faisait partie dela compagnie
Ilareçuuncoupdecasquesurlatêteet futhospitalisé douzejoursàlhôpital U

In [40]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {
            "role": "system",
            "content": "Vous êtes un assistant très intelligent chargé de corriger et de réécrire les textes pour les rendre cohérents et grammaticalement corrects"
        },
        {
            "role": "user",
            "content": f"Corrigez et réécrivez ce texte en francais en utilisant tout les mots dans l'ordre: {text_to_process}"
        }

    ],
  temperature=1,
  max_tokens=1000,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)



In [41]:
print("After processing:\n")
print_text(response.choices[0].message.content)


After processing:

Les derniers témoins à charge à Genève en mai, M. Cogne a entendu le novembre le
discours de Nicola, un discours violent dont il a retenu cette phrase : "Cette
vieille prostituée dar'mée". Un peu plus tard, il a également entendu un orateur
dire : "Enfonçons le barrage". Le témoin est assermenté. Cogne se trouvait au
carrefour à l'arrivée de la troupe. Il a vu un manifestant lancer du poivre aux
soldats et à un garde-frontière. Le témoin a vu des individus qui frappaient des
soldats. M. Augustin Gaille, agent de police à Lausanne, a assisté le jeudi 1er
novembre à un meeting tenu à Lausanne au cours duquel Lucien Tronchet a commenté
les événements de Genève. Tronchet a notamment dit que "pour un qui a fait du
service militaire, il n'y avait pas à se tromper sur la signification du
clairon". Tronchet a ajouté que pour son compte, "Yil s'était caché derrière une
automobile". MM. Duflon et Muller ont vu l'accusé M. Dupont marcher au premier
rang d'une colonne de manifes